Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

El siguiente filtro consiste en añadir una imagen a nuestro rostro.



In [3]:
normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()
font = cv2.FONT_HERSHEY_SIMPLEX

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

imodoF = 0
imodoE = 0

debug = 0

cap.set(3, 640)
cap.set(4, 480)

while True:
    t = time.time()
    ret, frame = cap.read()

    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            [lex, ley, rex, rey] = eyes
            if lex > -1:
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)

                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

                filtro = cv2.imread('filtro.png', -1)
                filtro_ancho = w
                filtro_alto = h

                if filtro.shape[2] == 4:
                    
                    #Modificamos las dimenciones del filtro 
                    filtro_resized = cv2.resize(filtro, (w, h))
                    
                    for c in range(0, 3):
                        
                        # Combina los canales del filtro y la imagen de la cara 
                        frame[y:y+h, x:x+w, c] = frame[y:y+h, x:x+w, c] * \
                            (1 - filtro_resized[:, :, 3] / 255.0) + \
                            filtro_resized[:, :, c] * \
                            (filtro_resized[:, :, 3] / 255.0)
                else:
                    frame[y:y+h, x:x+w] = cv2.addWeighted(frame[y:y+h, x:x+w], 1, filtro, 0.5, 0)

                cv2.imshow("Normalized", NormBGR)

    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)

    tec = cv2.waitKey(40)
    if tec & tec == 27:
        break

cap.release()
cv2.destroyAllWindows()

Camera 0
